In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from os import system
import os
import keras
import util
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from sklearn.cluster import KMeans

Using TensorFlow backend.


In [3]:
import xml.etree.ElementTree as ET
import classification_starter as start
import os
from collections import Counter
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET
import numpy as np
from scipy import sparse

import util

In [6]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from gensim import corpora, models, similarities
import gensim

In [9]:
labels = []
with open('t_train_starter.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        labels.append(row)
        
sentences = []
sent_lengths = []
with open('sentences.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        sent_lengths.append(len(row))
        sentences.append(row)

In [10]:
np.array(labels[0]).astype(int)

array([ 8,  6, 12, ...,  8,  8,  3])

In [15]:
ls = len(sentences)
#lv = len(sentences_valid)
y_train = np.array(labels[0][:ls]).astype(int)
#y_valid = np.array(labels[0][ls:ls+lv]).astype(int)

In [16]:
system("say I am finished")

0

In [41]:
# Set values for various parameters
num_features = 5000    # Word vector dimensionality                      
min_word_count = 10   # Minimum word count                        
num_workers = 6       # Number of threads to run in parallel
context = 15          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

2017-03-08 20:06:00,793 : INFO : collecting all words and their counts
2017-03-08 20:06:00,796 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


 Training model...


2017-03-08 20:06:22,383 : INFO : collected 1121376 word types from a corpus of 77704229 raw words and 3086 sentences
2017-03-08 20:06:22,384 : INFO : Loading a fresh vocabulary
2017-03-08 20:06:24,648 : INFO : min_count=10 retains 39527 unique words (3% of original 1121376, drops 1081849)
2017-03-08 20:06:24,650 : INFO : min_count=10 leaves 76154919 word corpus (98% of original 77704229, drops 1549310)
2017-03-08 20:06:24,850 : INFO : deleting the raw counts dictionary of 1121376 items
2017-03-08 20:06:24,909 : INFO : sample=0.001 downsamples 58 most-common words
2017-03-08 20:06:24,911 : INFO : downsampling leaves estimated 55404551 word corpus (72.8% of prior 76154919)
2017-03-08 20:06:24,912 : INFO : estimated required memory for 39527 words and 5000 dimensions: 1600843500 bytes
2017-03-08 20:06:25,093 : INFO : resetting layer weights
2017-03-08 20:06:31,050 : INFO : training model with 6 workers on 39527 vocabulary and 5000 features, using sg=0 hs=0 sample=0.001 negative=5 window=1

In [42]:
model.wv.syn0.shape

(39527, 5000)

In [43]:
# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] / 150

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )


# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [44]:
from os import system
system("say I am finished")

0

In [45]:
# For the first 3 clusters
for cluster in xrange(0,3):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
['"nt-Type:', '"02&amp;ticksBoot', '/tba/cu', 'gzip.."', '(compatible;"', 'HTTP/1.1"', '696CB5F7"', 'Web', 'We"', '"ntent-Type:', 'v1.0..Host:', '"ction:', 'gzip..Hos"', '"-Agent:', 'hello&amp;jedi', '"tion/x-www-form-"', '0', 'DSP', '"219"', 'W"', 'COM"', 'Fri,', '/bins/int/9k"', 'Yet', '*/*"', '336CA6489"', 'GM"', 'E17B"', '".Content-Type:', 'Mozil"', '"oding:', '"{85CB6900-4D95-11CF-960C-0080C7F4EE85}"', '864422066720&amp;c"', 'Server..Da"', 'Anothe"', '"&amp;session', 'Wed,', '/abt?udata', 'Aug', '"sAlive', 'Yaws/1.68', '"Host:', '6."', '/;"', 'BUS', '"a3edbe9053889fb24ab22492472b39dc"', '"h:', '"he', 'href', '"upd.host255-255-"', 'DEVa', '0F1"', '"ersion', '"ccept-Encoding:', 'Ap"', '"B5&amp;ticksBoot', 'HTTP"', '"2009', 'path', '"ser-Agent:', '"e', '"n', '"t', '"p', '"r', '".', '":', '/jedi?reques"', '"tiveWindows', '100"', 'clo"', '"261"', '"/1.1..Content-Le"', '"C:\\WINDOWS\\system32\\winmm.dll"', '"ug', 'CURa', '/>\n<message>OverlappedWinsockIO_Send</message>\n<send

In [49]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    num_centroids = max( word_centroid_map.values() ) + 1

    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )

    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1

    return bag_of_centroids

In [51]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (len(sentences), num_clusters), \
    dtype="float32" )

# Transform the training set into bags of centroids
counter = 0
for sentence in sentences:
    train_centroids[counter] = create_bag_of_centroids( sentence, \
        word_centroid_map )
    counter += 1

KeyboardInterrupt: 

In [ ]:
test_centroids = np.zeros(( 3724, num_clusters), \
    dtype="float32" )

counter = 0
with open('sentences_test.csv') as csvfile:
    reader = csv.reader(csvfile)
    for sentence in reader:
        test_centroids[counter] = create_bag_of_centroids( sentence, \
            word_centroid_map )
        counter += 1
    
train_centroids=train_centroids.astype(int)
test_centroids=test_centroids.astype(int)

In [ ]:
valid_centroids = np.zeros(( len(sentences_valid), num_clusters), \
    dtype="float32" )

counter = 0
for sentence in sentences_valid:
    valid_centroids[counter] = create_bag_of_centroids( sentence, \
        word_centroid_map )
    counter += 1
    
train_centroids=train_centroids.astype(int)
valid_centroids=valid_centroids.astype(int)

In [ ]:
sentences_test = []

with open('sentences_test.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:

        sentences_test.append(row)

In [26]:
print test_centroids.shape
print train_centroids.shape

NameError: name 'test_centroids' is not defined

In [27]:
valid_centroids

NameError: name 'valid_centroids' is not defined

In [28]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train,15)
y_valid = to_categorical(y_valid,15)

NameError: name 'y_valid' is not defined

In [29]:
net = Sequential()
net.add(Dense(57, input_dim=200))

net.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [30]:
net.fit(train_centroids, y_train, nb_epoch=5)

NameError: name 'train_centroids' is not defined

In [52]:
print train_centroids.shape
print y_train.shape

In [32]:
ids = pd.read_csv("test_ids.csv")
ids

IOError: File test_ids.csv does not exist

In [33]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 200)

y_train = np.array(labels[0]).astype(int)

# Fitting the forest may take a few minutes
print "Fitting a random forest to labeled training data..."
forest = forest.fit(train_centroids,y_train)
result = forest.predict(test_centroids)

Fitting a random forest to labeled training data...


NameError: name 'train_centroids' is not defined

In [34]:
df_res = pd.DataFrame(ids)
df_res["Prediction"] = result
df_res.columns = ['Id','Prediction']
df_res

NameError: name 'ids' is not defined

In [35]:
ids = pd.read_csv("train_ids_starter.csv",header=None)

IOError: File train_ids_starter.csv does not exist

In [36]:
ids = np.array(ids)

NameError: name 'ids' is not defined

In [37]:
df_res.to_csv("rand_forest_preds2.csv",index=None)

NameError: name 'df_res' is not defined

In [38]:
p = pd.read_csv("rand_forest_preds1.csv")
p

IOError: File rand_forest_preds1.csv does not exist

In [39]:
print "Wrong classifications:", np.count_nonzero(y_valid-result), " of ", len(y_valid-result)

Wrong classifications:

NameError: name 'y_valid' is not defined

In [40]:
train_centroids

NameError: name 'train_centroids' is not defined